# Context API - Solutions

Complete implementations for all exercises.

In [ ]:
import sys
sys.path.insert(0, r'c:\Users\hodibi\OneDrive - Ingredion\Desktop\Repos\Odibi')

from odibi.context import Context, PandasContext
import pandas as pd
from typing import Dict, Any, List, Callable, Set
import pickle
from pathlib import Path

## Solution 1: Snapshot Context

In [ ]:
class SnapshotContext(PandasContext):
    """Context with named snapshot capability."""
    
    def __init__(self):
        super().__init__()
        self._snapshots: Dict[str, Dict[str, pd.DataFrame]] = {}
    
    def snapshot(self, name: str) -> None:
        """Save current state with name."""
        # Deep copy to avoid reference issues
        self._snapshots[name] = {k: v.copy() for k, v in self._data.items()}
    
    def restore(self, name: str) -> None:
        """Restore to named snapshot."""
        if name not in self._snapshots:
            available = ", ".join(self._snapshots.keys()) if self._snapshots else "none"
            raise KeyError(f"Snapshot '{name}' not found. Available: {available}")
        
        # Restore by copying snapshot data
        self._data = {k: v.copy() for k, v in self._snapshots[name].items()}
    
    def list_snapshots(self) -> List[str]:
        """List available snapshots."""
        return list(self._snapshots.keys())

# Test
ctx = SnapshotContext()
ctx.register('a', pd.DataFrame({'x': [1, 2]}))
ctx.snapshot('initial')

ctx.register('b', pd.DataFrame({'y': [3, 4]}))
ctx.snapshot('with_b')

ctx.register('c', pd.DataFrame({'z': [5, 6]}))
print("Current:", ctx.list_names())
print("Snapshots:", ctx.list_snapshots())

ctx.restore('initial')
print("After restore:", ctx.list_names())
assert ctx.list_names() == ['a']
print("✅ Solution 1 works!")

### Key Points
- Use `copy()` to avoid shared references
- Store snapshots in a dict for named access
- Provide helpful error messages with available snapshots

## Solution 2: Serializable Context

In [ ]:
class SerializableContext(PandasContext):
    """Context that can be saved/loaded from disk."""
    
    def save(self, path: str) -> None:
        """Save context to file."""
        with open(path, 'wb') as f:
            pickle.dump(self._data, f)
    
    def load(self, path: str) -> None:
        """Load context from file."""
        if not Path(path).exists():
            raise FileNotFoundError(f"Context file not found: {path}")
        
        with open(path, 'rb') as f:
            self._data = pickle.load(f)

# Test
ctx = SerializableContext()
ctx.register('data', pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]}))

ctx.save('test_context.pkl')
print("Saved:", ctx.list_names())

ctx2 = SerializableContext()
ctx2.load('test_context.pkl')
print("Loaded:", ctx2.list_names())
assert ctx2.get('data').equals(ctx.get('data'))

Path('test_context.pkl').unlink()
print("✅ Solution 2 works!")

### Key Points
- Pickle handles Pandas DataFrames well
- Check file existence before loading
- Remember to use binary mode ('wb'/'rb')

### Alternative: Parquet for production
For real systems, consider saving each DataFrame as Parquet + metadata JSON.

## Solution 3: Observable Context

In [ ]:
class ObservableContext(PandasContext):
    """Context that notifies observers of changes."""
    
    def __init__(self):
        super().__init__()
        self._observers: Set[Callable[[str, str | None], None]] = set()
    
    def subscribe(self, callback: Callable[[str, str | None], None]) -> None:
        """Add observer."""
        self._observers.add(callback)
    
    def unsubscribe(self, callback: Callable[[str, str | None], None]) -> None:
        """Remove observer."""
        self._observers.discard(callback)
    
    def _notify(self, event: str, name: str | None = None) -> None:
        """Notify all observers."""
        for observer in self._observers:
            observer(event, name)
    
    def register(self, name: str, df: pd.DataFrame) -> None:
        """Register and notify."""
        super().register(name, df)
        self._notify('register', name)
    
    def clear(self) -> None:
        """Clear and notify."""
        super().clear()
        self._notify('clear', None)

# Test
events = []

def logger(event: str, name: str | None):
    events.append((event, name))
    print(f"📢 {event}: {name}")

ctx = ObservableContext()
ctx.subscribe(logger)

ctx.register('df1', pd.DataFrame({'x': [1]}))
ctx.register('df2', pd.DataFrame({'y': [2]}))
ctx.clear()

assert events == [('register', 'df1'), ('register', 'df2'), ('clear', None)]
print("✅ Solution 3 works!")

### Key Points
- Use `set()` to avoid duplicate observers
- Call `super()` methods first, then notify
- Observer pattern enables logging, metrics, debugging

### Use Cases
- Audit logging: Track all data registrations
- Metrics: Count context operations
- Debugging: Print context changes in dev

## Solution 4: Validating Context

In [ ]:
import numpy as np

class ValidatingContext(PandasContext):
    """Context with schema validation."""
    
    def __init__(self):
        super().__init__()
        self._schemas: Dict[str, Dict[str, type]] = {}
    
    def set_schema(self, name: str, schema: Dict[str, type]) -> None:
        """Define expected schema for a DataFrame."""
        self._schemas[name] = schema
    
    def _validate_schema(self, name: str, df: pd.DataFrame) -> None:
        """Check if DataFrame matches schema."""
        if name not in self._schemas:
            return  # No schema defined, skip validation
        
        schema = self._schemas[name]
        
        # Check for missing columns
        missing = set(schema.keys()) - set(df.columns)
        if missing:
            raise ValueError(f"Missing columns: {missing}")
        
        # Check types
        for col, expected_type in schema.items():
            actual_type = df[col].dtype
            if actual_type != expected_type:
                raise ValueError(
                    f"Column '{col}' has type {actual_type}, expected {expected_type}"
                )
    
    def register(self, name: str, df: pd.DataFrame) -> None:
        """Register with validation."""
        self._validate_schema(name, df)
        super().register(name, df)

# Test
ctx = ValidatingContext()
ctx.set_schema('customers', {
    'customer_id': np.int64,
    'name': object
})

# Valid DataFrame
valid_df = pd.DataFrame({
    'customer_id': [1, 2, 3],
    'name': ['Alice', 'Bob', 'Carol']
})
ctx.register('customers', valid_df)
print("✅ Valid DataFrame registered")

# Invalid: missing column
try:
    ctx.register('customers', pd.DataFrame({'customer_id': [1]}))
    assert False, "Should have raised ValueError"
except ValueError as e:
    print(f"❌ Caught: {e}")

# Invalid: wrong type
try:
    invalid_df = pd.DataFrame({
        'customer_id': ['a', 'b'],
        'name': ['Alice', 'Bob']
    })
    ctx.register('customers', invalid_df)
    assert False, "Should have raised ValueError"
except ValueError as e:
    print(f"❌ Caught: {e}")

print("✅ Solution 4 works!")

### Key Points
- Validate before calling `super().register()`
- Check both missing columns and type mismatches
- Skip validation if no schema defined

### Production Enhancements
- Use `pandera` or `great_expectations` for richer validation
- Support nullable columns
- Validate ranges, patterns, uniqueness constraints

## Bonus: SuperContext

Combines LRU caching, snapshots, and observability.

In [ ]:
from collections import OrderedDict

class SuperContext(Context):
    """Context with LRU caching, snapshots, and observers."""
    
    def __init__(self, max_size: int = 10):
        self.max_size = max_size
        self._data: OrderedDict[str, pd.DataFrame] = OrderedDict()
        self._snapshots: Dict[str, Dict[str, pd.DataFrame]] = {}
        self._observers: Set[Callable[[str, str | None], None]] = set()
        self._eviction_count = 0
    
    # Observer methods
    def subscribe(self, callback: Callable[[str, str | None], None]) -> None:
        self._observers.add(callback)
    
    def _notify(self, event: str, name: str | None = None) -> None:
        for observer in self._observers:
            observer(event, name)
    
    # Snapshot methods
    def snapshot(self, name: str) -> None:
        # Note: Don't snapshot observers (implementation choice)
        self._snapshots[name] = {k: v.copy() for k, v in self._data.items()}
        self._notify('snapshot', name)
    
    def restore(self, name: str) -> None:
        if name not in self._snapshots:
            raise KeyError(f"Snapshot '{name}' not found")
        self._data = OrderedDict({k: v.copy() for k, v in self._snapshots[name].items()})
        self._notify('restore', name)
    
    # Context ABC methods (with LRU)
    def register(self, name: str, df: pd.DataFrame) -> None:
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"Expected pandas.DataFrame, got {type(df)}")
        
        if name in self._data:
            del self._data[name]
        
        # LRU eviction
        if len(self._data) >= self.max_size:
            evicted_name, _ = self._data.popitem(last=False)
            self._eviction_count += 1
            self._notify('evict', evicted_name)
        
        self._data[name] = df
        self._notify('register', name)
    
    def get(self, name: str) -> pd.DataFrame:
        if name not in self._data:
            raise KeyError(f"DataFrame '{name}' not found")
        self._data.move_to_end(name)
        self._notify('get', name)
        return self._data[name]
    
    def has(self, name: str) -> bool:
        return name in self._data
    
    def list_names(self) -> List[str]:
        return list(self._data.keys())
    
    def clear(self) -> None:
        self._data.clear()
        self._eviction_count = 0
        self._notify('clear', None)

# Test combined features
events = []
ctx = SuperContext(max_size=2)
ctx.subscribe(lambda e, n: events.append((e, n)))

ctx.register('a', pd.DataFrame({'x': [1]}))
ctx.snapshot('s1')
ctx.register('b', pd.DataFrame({'x': [2]}))
ctx.register('c', pd.DataFrame({'x': [3]}))  # Evicts 'a'

print("Events:", events)
print("Current:", ctx.list_names())

ctx.restore('s1')
print("After restore:", ctx.list_names())
print("✅ SuperContext works!")

### Design Decisions
1. **Snapshots don't include observers** - observers are runtime concerns, snapshots are data
2. **Evictions trigger events** - helps debug cache misses
3. **OrderedDict for LRU** - simpler than separate tracking

### Trade-offs
- More features = more complexity
- Consider composition over inheritance for production
- Document feature interactions clearly